# Project 4: West Nile Virus Prediction
## Predict West Nile virus in mosquitos across the city of Chicago


----

## Background 

West Nile virus is most commonly spread to humans through infected mosquitos. Around 20% of people who become infected with the virus develop symptoms ranging from a persistent fever, to serious neurological illnesses that can result in death.

In 2002, the first human cases of West Nile virus were reported in Chicago. No vaccine or specific medicines are available for the West Nile virus infection. Due to the serious health implications and cost of seeking treatment,it became a pressing concern that required intervention by the state. By 2004 the City of Chicago and the Chicago Department of Public Health (CDPH) had established a comprehensive surveillance and control program that is still in effect today.

Every week from late spring through the fall, mosquitos in traps across the city are tested for the virus. The results of these tests influence when and where the city will spray airborne pesticides to control adult mosquito populations.

## Problem Statement

However, it is a huge cost to the state to carry out the planned sprays so as a result, spraying where it has the highest impact and spread is of utmost importance. The Centre of Disease Control and Prevention (CDC) hired our team of data scientists to better understand and evaluate where and when the pesticide should be sprayed given a number of variables.

Given weather, trap locations and spray data, we were tasked to predict when and where different species of mosquitos will test positive for West Nile virus. We aimed to develop a model that could accurately predict outbreaks of West Nile virus in mosquitos such that it would help the City of Chicago and CPHD more efficiently and effectively in allocating resources towards preventing transmission of this potentially deadly virus. 

### Contents

1. Data Cleaning
2. Exploratory Data Analysis
3. Model Selection
4. Conclusion and Recommendations

## 1.Data Cleaning

In [1]:
# Import libraries 
import numpy as np
import pandas as pd


# Others
import time
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100) # Enables Pandas to display long strings properly

import warnings 
warnings.filterwarnings('ignore')

In [2]:
# Load datasets
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df_spray = pd.read_csv('../data/spray.csv')
df_weather = pd.read_csv('../data/weather.csv')

### 1.1.Train

In [3]:
# Display first 5 rows of dataset
df_train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [4]:
# Display shape
df_train.shape

(10506, 12)

In [5]:
# Get info & dtypes
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    10506 non-null  object 
 1   Address                 10506 non-null  object 
 2   Species                 10506 non-null  object 
 3   Block                   10506 non-null  int64  
 4   Street                  10506 non-null  object 
 5   Trap                    10506 non-null  object 
 6   AddressNumberAndStreet  10506 non-null  object 
 7   Latitude                10506 non-null  float64
 8   Longitude               10506 non-null  float64
 9   AddressAccuracy         10506 non-null  int64  
 10  NumMosquitos            10506 non-null  int64  
 11  WnvPresent              10506 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


Data types for each column as expected

In [6]:
# Describe dataset
df_train.describe()

,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000


No anomalies spotted in the min and max values. As expected, since WnvPresent is a binary indicator, the min is 0 and max is 1. For NumMosquitos, given that the data entry is such that only a maximum of 50 mosquitos can be recorded in a row, it is not surprising to see max of 50

In [7]:
# Check for columns with null values
df_train.isnull().sum()

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64

No null values to remove or replace

In [8]:
# Drop 'Address', 'Block', 'Street', 'AddressNumberAndStreet' and 'AddressAccuracy' columns as we already have the 'Latitude' and 'Longitude' columns
df_train.drop(columns=['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'], inplace=True)

In [9]:
# Check for duplicates between col_names
# test results are organized in such a way that when the number of mosquitos exceed 50\
# they are split into another record (another row in the dataset)\
# we should aggregate rows for the same date, species, location, trap

col_names = ['Date', 'Species', 'Trap', 'Latitude','Longitude']
duplicates = df_train.duplicated(subset= col_names, keep = False)
df_train[duplicates].head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent
98,2007-06-26,CULEX PIPIENS/RESTUANS,T086,41.688324,-87.676709,1,0
99,2007-06-26,CULEX PIPIENS/RESTUANS,T086,41.688324,-87.676709,1,0
280,2007-07-11,CULEX RESTUANS,T015,41.974089,-87.824812,1,0
281,2007-07-11,CULEX RESTUANS,T015,41.974089,-87.824812,2,0
293,2007-07-11,CULEX PIPIENS/RESTUANS,T086,41.688324,-87.676709,50,0


In [10]:
#check number of duplicated rows
df_train[duplicates].shape

(2786, 7)

In [11]:
# Collapse rows groupby col_names and summing 'NumMosquitos' and 'WnvPresent'
df_train = df_train.groupby(by=col_names).sum().reset_index()
df_train.head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent
0,2007-05-29,CULEX PIPIENS,T096,41.731922,-87.677512,1,0
1,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0
2,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,1,0
4,2007-05-29,CULEX PIPIENS/RESTUANS,T050,41.919343,-87.694259,1,0


In [12]:
# Remap 'WnvPresent' such that any rows within same date/species/trap/latitude/longitude has a presence of WNV will be assigned 1, otherwise 0.
df_train['WnvPresent'] = df_train['WnvPresent'].map(lambda x: 1 if x > 0 else 0)

In [13]:
# Create datetime metrics
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_train['Year'] = df_train['Date'].apply(lambda x: x.year)
df_train['Month'] = df_train['Date'].apply(lambda x: x.month)
df_train['Week'] = df_train['Date'].apply(lambda x: x.week)
df_train['DayOfWeek'] = df_train['Date'].apply(lambda x: x.dayofweek)

In [14]:
# Preview dataset after cleanup
df_train.head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Year,Month,Week,DayOfWeek
0,2007-05-29,CULEX PIPIENS,T096,41.731922,-87.677512,1,0,2007,5,22,1
1,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0,2007,5,22,1
2,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0,2007,5,22,1
3,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,1,0,2007,5,22,1
4,2007-05-29,CULEX PIPIENS/RESTUANS,T050,41.919343,-87.694259,1,0,2007,5,22,1


In [15]:
#check the size and stats of df_train after the cleanup to ensure nothing went wrong
print(df_train.shape)
df_train.describe()

(8475, 11)


,Latitude,Longitude,NumMosquitos,WnvPresent,Year,Month,Week,DayOfWeek
count,8475.000000,8475.000000,8475.000000,8475.000000,8475.000000,8475.000000,8475.000000,8475.000000
mean,41.844618,-87.694788,15.933805,0.053923,2009.713156,7.700531,31.739469,2.659233
std,0.106609,0.084063,75.084707,0.225880,2.331751,1.108677,4.717594,1.398601
min,41.644612,-87.930995,1.000000,0.000000,2007.000000,5.000000,22.000000,0.000000
25%,41.750498,-87.752329,2.000000,0.000000,2007.000000,7.000000,28.000000,2.000000
50%,41.857402,-87.694991,4.000000,0.000000,2009.000000,8.000000,32.000000,3.000000
75%,41.944869,-87.642984,11.000000,0.000000,2011.000000,9.000000,35.000000,4.000000
max,42.017430,-87.531635,2532.000000,1.000000,2013.000000,10.000000,41.000000,4.000000


After collapsing the rows with the same attributes (all except nummosquitos and wnv present), we have reduced the number of rows from 10506 to 8475, and have a more accurate stats (mean, max) of NumMosquitos per location, date and species.

WnvPresent is also correctly taking a binary form of 0 (i.e. none of mosquitoes from a particular species taken in that location and date had Wnv) or 1 (i.e. any one mosquito from a particular species taken in that location and date had Wnv)

In [16]:
# Save data
df_train.to_csv('../data/train_clean.csv', index = False)

### 1.2.Spray

In [17]:
# Display first 5 rows of dataset
df_spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [18]:
# Display shape
df_spray.shape

(14835, 4)

In [19]:
# Get info & dtypes
df_spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14835 non-null  object 
 1   Time       14251 non-null  object 
 2   Latitude   14835 non-null  float64
 3   Longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [20]:
# Describe dataset
df_spray.describe()

,Latitude,Longitude
count,14835.000000,14835.000000
mean,41.904828,-87.736690
std,0.104381,0.067292
min,41.713925,-88.096468
25%,41.785001,-87.794225
50%,41.940075,-87.727853
75%,41.980978,-87.694108
max,42.395983,-87.586727


In [21]:
# Check for columns with null values
df_spray.isnull().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

Time has null values, but it is not an important feature for our project (it is not a variable that we are testing for its importance to the presence of West Nile Virus), hence we will drop it

In [22]:
# Time is not an important feature; we will drop it
df_spray.drop(columns='Time', inplace=True)

In [23]:
# Check for number of exact duplicates
df_spray.duplicated().sum()

541

In [24]:
# Check for exact duplicates
df_spray[df_spray.duplicated()]

,Date,Latitude,Longitude
485,2011-09-07,41.983917,-87.793088
490,2011-09-07,41.986460,-87.794225
491,2011-09-07,41.986460,-87.794225
492,2011-09-07,41.986460,-87.794225
493,2011-09-07,41.986460,-87.794225
494,2011-09-07,41.986460,-87.794225
495,2011-09-07,41.986460,-87.794225
496,2011-09-07,41.986460,-87.794225
497,2011-09-07,41.986460,-87.794225
498,2011-09-07,41.986460,-87.794225


There are also no known issues with the way data is collected (unlike the number of mosquitoes in the train set above, where duplicate rows are due to the max NumMosquito restriction, and actually contain important information). As we are only concerned if there was spraying in a location on a certain date, duplicate entries in the dataset will not be useful and we will drop them

In [25]:
# Drop exact duplicates
df_spray.drop_duplicates(inplace=True)

In [26]:
# Create datetime metrics
df_spray['Date'] = pd.to_datetime(df_spray['Date'])
df_spray['Year'] = df_spray['Date'].apply(lambda x: x.year)
df_spray['Month'] = df_spray['Date'].apply(lambda x: x.month)
df_spray['Week'] = df_spray['Date'].apply(lambda x: x.week)
df_spray['DayOfWeek'] = df_spray['Date'].apply(lambda x: x.dayofweek)

In [27]:
# Preview dataset
df_spray.head()

,Date,Latitude,Longitude,Year,Month,Week,DayOfWeek
0,2011-08-29,42.391623,-88.089163,2011,8,35,0
1,2011-08-29,42.391348,-88.089163,2011,8,35,0
2,2011-08-29,42.391022,-88.089157,2011,8,35,0
3,2011-08-29,42.390637,-88.089158,2011,8,35,0
4,2011-08-29,42.390410,-88.088858,2011,8,35,0


In [28]:
# Save data
df_spray.to_csv('../data/spray_clean.csv', index = False)

### 1.3.Weather

In [29]:
# Display first 5 rows of dataset
df_weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [30]:
# Display shape
df_weather.shape

(2944, 22)

In [31]:
# Get info & dtypes
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

In [32]:
# Describe dataset
df_weather.describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


In [33]:
#From preview of dataset, CodeSum appears to have empty string columns
df_weather['CodeSum'].value_counts().head()

              1609
RA             296
RA BR          238
BR             110
TSRA RA BR      92
Name: CodeSum, dtype: int64

CodeSum has 1609 rows with empty strings

In [34]:
# Replace all empty strings with Nan (i.e. empty strings noted in 'CodeSum' column)
df_weather.replace(r'^\s*$', np.NaN, regex=True, inplace=True)

In [35]:
df_weather['CodeSum'].value_counts().head()

RA            296
RA BR         238
BR            110
TSRA RA BR     92
BR HZ          81
Name: CodeSum, dtype: int64

In [36]:
# Check for columns with null values
df_weather.isnull().sum().sort_values(ascending=False)



CodeSum        1609
Station           0
Date              0
ResultDir         0
ResultSpeed       0
SeaLevel          0
StnPressure       0
PrecipTotal       0
SnowFall          0
Water1            0
Depth             0
Sunset            0
Sunrise           0
Cool              0
Heat              0
WetBulb           0
DewPoint          0
Depart            0
Tavg              0
Tmin              0
Tmax              0
AvgSpeed          0
dtype: int64

CodeSum gives the weather types (e.g. Tornado, Thunderstorm). While it could be potentially useful in our project, it has >50% values that are null. There are also other useful columns in the data set that gives temperature and precipitation already, hence the information loss from dropping this column should be minimal. 

In [37]:
# Check for number of exact duplicates
df_weather.duplicated().sum()

0

In [38]:
# Check for'M' values as they are missing values
m_values = df_weather.isin(['M']).sum()
m_values = m_values[m_values>0].sort_values(ascending=False)
m_values

Water1         2944
Depart         1472
Depth          1472
SnowFall       1472
Tavg             11
Heat             11
Cool             11
SeaLevel          9
WetBulb           4
StnPressure       4
AvgSpeed          3
PrecipTotal       2
dtype: int64

In [39]:
# Find count of 'M' values as a percentage total number of rows 
for key, value in m_values.items():
    x = round(value/len(df_weather)*100,2)
    print (f'{key}: {value}, {x}%')
    


Water1: 2944, 100.0%
Depart: 1472, 50.0%
Depth: 1472, 50.0%
SnowFall: 1472, 50.0%
Tavg: 11, 0.37%
Heat: 11, 0.37%
Cool: 11, 0.37%
SeaLevel: 9, 0.31%
WetBulb: 4, 0.14%
StnPressure: 4, 0.14%
AvgSpeed: 3, 0.1%
PrecipTotal: 2, 0.07%


Water1 column has 100% missing data. It is believed that dry conditions are more favorable for West Nile virus than wet. However, there is a column indicating precipitation, which will be more important for our study, so dropping the Water1 column should not lead to loss of important data

In [40]:
# Check for 'T' values as they are trace values, i.e. very little close to zero
t_values = df_weather.isin(['  T']).sum()
t_values = t_values[t_values>0].sort_values(ascending = False)
t_values

PrecipTotal    318
SnowFall        12
dtype: int64

In [41]:
# Find count of 'T' values as a percentage total number of rows 
for key, value in t_values.items():
    x = round(value/len(df_weather)*100,2)
    print (f'{key}: {value}, {x}%')


PrecipTotal: 318, 10.8%
SnowFall: 12, 0.41%


In [42]:
# Check values in 'Depth' column, as we previously saw there was 50% missing information
df_weather['Depth'].value_counts()


M    1472
0    1472
Name: Depth, dtype: int64

In [43]:
# Check values in 'SnowFall' column, as we previously saw there was 50% missing information
df_weather['SnowFall'].value_counts()


M      1472
0.0    1459
  T      12
0.1       1
Name: SnowFall, dtype: int64

As the snowfall and snowdepth have a lot of missing information, zero, or trace data, we will drop these columns. Additionally, there are no studies that snow specifically affects mosquito breeding--although cold weather (alreasy represented by temperature) is known to reduce breeding. 

### Dropping/ Imputing Features

Summary of our steps:
- Drop 'Water1', 'Depth', 'SnowFall'and 'CodeSum' columns due to the high % of missing values. There will be no loss of important data as well, since related information on temperature and precipitation are present in other columns
- Drop 'Depart' column because only Station 1 has readings. The data may also not be useful in any case, since we already have temperature readings (Tavg, Tmin, Tmax).
- Impute null values for 'Tavg' with the average of 'Tmax' and 'Tmin'.
- Replace 'T' in 'PrecipTotal' with '0'.
- Apply forward filling method for null values in 'Cool', 'Heat', 'SeaLevel', 'WetBulb', 'StnPressure', 'AvgSpeed' and 'PrecipTotal'(i.e. previous day's readings in the respective Stations, as there is likely a high autocorrelation)
- Impute 'Sunrise' and 'Sunset' for Station 2 using Station 1's values. 

In [44]:
# Drop columns 
df_weather = df_weather.drop(columns= ['Water1', 'Depth', 'SnowFall', 'CodeSum', 'Depart'])

In [45]:
# Replace 'M' with Nan for imputation later and 'T' with '0'
def fill_null_zero(df, columns):
    for i in columns:
        df[i] = df[i].apply(lambda x: np.nan if x =='M' else x)
        df[i] = df[i].apply(lambda x: 0 if x =='  T' else x)       
        df[i] = df[i].astype('float')

In [46]:
# Columns that we want to impute
cols = ['Tavg','Cool', 'Heat', 'SeaLevel', 'WetBulb', 'StnPressure', 'AvgSpeed', 'PrecipTotal']
fill_null_zero(df_weather, cols) 

'Tavg' that have missing values, have corresponding 'Tmax' and 'Tmin' values. Hence, we can estimate its 'Tavg' by taking the average of these values.

In [47]:
# Impute null values for 'Tavg'
df_weather['Tavg'] =  df_weather['Tavg'].fillna(round((df_weather['Tmax'] - df_weather['Tmin'])/2))

In [48]:
# Check remaining null values
df_weather.isnull().sum().sort_values(ascending=False)

Cool           11
Heat           11
SeaLevel        9
WetBulb         4
StnPressure     4
AvgSpeed        3
PrecipTotal     2
DewPoint        0
Tavg            0
Date            0
Sunrise         0
Sunset          0
Tmin            0
Tmax            0
ResultSpeed     0
ResultDir       0
Station         0
dtype: int64

There are several ways to impute the null values for 'Cool', 'Heat', 'SeaLevel', 'WetBulb', 'StnPressure', 'AvgSpeed', and 'PrecipTotal'. One way is to impute with the monthly average figures for the respective stations. However, it may not be very precise because of the wide variance in each month. Imputing station 2's missing value with that of station 1 may not be ideal because they are located at different sites. Hence, we adopted the forward filling method for the null values, based on the assumption that there is some autocorrelation in the readings over two consecutive days, and taking into consideration the different stations. 

In [49]:
# Sort by Stations and perform forward fill for null values
df_weather.sort_values(['Station','Date'], inplace=True)
df_weather.fillna(method='ffill', inplace=True)

In [50]:
# Impute 'Sunrise' and 'Sunset' for Station 2 using Station 1's values as there should little variation in values despite the Stations' location
df_weather.loc[df_weather['Station'] == 2,'Sunrise'] = df_weather[df_weather['Station'] == 1]['Sunrise'].tolist()
df_weather.loc[df_weather['Station'] == 2,'Sunset'] = df_weather[df_weather['Station'] == 1]['Sunset'].tolist()

In [51]:
# Check remaining null values
df_weather.isnull().sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

### Feature Engineering

Summary of our steps:
- Added Year, Month, Week and Day of Week features
- Added Temperature Range ('Trange')

In [52]:
# Convert all columns to floats
for col in df_weather.columns:
    try:
        df_weather[col] = df_weather[col].astype(float)
    except:
        print(col, 'cannot be transformed into float')
        pass

Date cannot be transformed into float


In [53]:
# Create datetime metrics
df_weather['Date'] = pd.to_datetime(df_weather['Date'])
df_weather['Year'] = df_weather['Date'].apply(lambda x: x.year)
df_weather['Month'] = df_weather['Date'].apply(lambda x: x.month)
df_weather['Week'] = df_weather['Date'].apply(lambda x: x.week)
df_weather['DayOfWeek'] = df_weather['Date'].apply(lambda x: x.dayofweek)

In [54]:
# Create new feature for temp range
df_weather['Trange'] = df_weather['Tmax'] - df_weather['Tmin']

In [55]:
# Preview dataset
df_weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month,Week,DayOfWeek,Trange
0,1.0,2007-05-01,83.0,50.0,67.0,51.0,56.0,0.0,2.0,448.0,1849.0,0.0,29.10,29.82,1.7,27.0,9.2,2007,5,18,1,33.0
2,1.0,2007-05-02,59.0,42.0,51.0,42.0,47.0,14.0,0.0,447.0,1850.0,0.0,29.38,30.09,13.0,4.0,13.4,2007,5,18,2,17.0
4,1.0,2007-05-03,66.0,46.0,56.0,40.0,48.0,9.0,0.0,446.0,1851.0,0.0,29.39,30.12,11.7,7.0,11.9,2007,5,18,3,20.0
6,1.0,2007-05-04,66.0,49.0,58.0,41.0,50.0,7.0,0.0,444.0,1852.0,0.0,29.31,30.05,10.4,8.0,10.8,2007,5,18,4,17.0
8,1.0,2007-05-05,66.0,53.0,60.0,38.0,49.0,5.0,0.0,443.0,1853.0,0.0,29.40,30.10,11.7,7.0,12.0,2007,5,18,5,13.0


In [56]:
df_weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month,Week,DayOfWeek,Trange
0,1.0,2007-05-01,83.0,50.0,67.0,51.0,56.0,0.0,2.0,448.0,1849.0,0.0,29.10,29.82,1.7,27.0,9.2,2007,5,18,1,33.0
2,1.0,2007-05-02,59.0,42.0,51.0,42.0,47.0,14.0,0.0,447.0,1850.0,0.0,29.38,30.09,13.0,4.0,13.4,2007,5,18,2,17.0
4,1.0,2007-05-03,66.0,46.0,56.0,40.0,48.0,9.0,0.0,446.0,1851.0,0.0,29.39,30.12,11.7,7.0,11.9,2007,5,18,3,20.0
6,1.0,2007-05-04,66.0,49.0,58.0,41.0,50.0,7.0,0.0,444.0,1852.0,0.0,29.31,30.05,10.4,8.0,10.8,2007,5,18,4,17.0
8,1.0,2007-05-05,66.0,53.0,60.0,38.0,49.0,5.0,0.0,443.0,1853.0,0.0,29.40,30.10,11.7,7.0,12.0,2007,5,18,5,13.0


In [57]:
# Save data
df_weather.to_csv('../data/weather_clean.csv', index = False)

## Merge `train_clean` with `weather_clean`

In [58]:
# import cleaned datasets
weather_clean = pd.read_csv('../data/weather_clean.csv')
train_clean = pd.read_csv('../data/train_clean.csv')

# coordinates for both stations
coord_S1 = (41.995, -87.933) 
coord_S2 = (41.786, -87.752) 

from geopy import distance
# for each row compare distance from each station using geopy
for i in range(len(train_clean)):
    coord = (train_clean['Latitude'][i],train_clean['Longitude'][i])
    distS1 = abs(distance.distance(coord_S1,coord).km)
    distS2 = abs(distance.distance(coord_S2,coord).km)
    # creating new column for distance from respective Station
    train_clean.loc[i,'DistFromS1']=round(distS1,1)
    train_clean.loc[i,'DistFromS2']=round(distS2,1)
    
    # station that is closest to the trap will be assigned accordingly
    if distS1<distS2:
        train_clean.loc[i,'Station']=1
    elif distS2<distS1:
        train_clean.loc[i,'Station']=2

# dropping duplicate columns on one of the two df
weather_clean.drop(columns = ['Year','Month','Week','DayOfWeek'], inplace=True)

In [59]:
# convert station no. from float to int
train_clean['Station'] = train_clean['Station'].astype(int)
weather_clean["Station"] = weather_clean["Station"].astype(int)

# converting date to datetime
train_clean['Date'] = pd.to_datetime(train_clean['Date'])
weather_clean['Date'] = pd.to_datetime(weather_clean['Date'])

# merge on Station and Date columns
df_combined = pd.merge(train_clean, weather_clean, on=['Station','Date'], how='left')
df_combined.head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Year,Month,Week,DayOfWeek,DistFromS1,DistFromS2,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Trange
0,2007-05-29,CULEX PIPIENS,T096,41.731922,-87.677512,1,0,2007,5,22,1,36.1,8.6,2,88.0,65.0,77.0,59.0,66.0,0.0,12.0,421.0,1917.0,0.0,29.44,30.09,5.8,16.0,7.4,23.0
1,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0,2007,5,22,1,11.8,19.2,1,88.0,60.0,74.0,58.0,65.0,0.0,9.0,421.0,1917.0,0.0,29.39,30.11,5.8,18.0,6.5,28.0
2,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0,2007,5,22,1,9.3,21.7,1,88.0,60.0,74.0,58.0,65.0,0.0,9.0,421.0,1917.0,0.0,29.39,30.11,5.8,18.0,6.5,28.0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,1,0,2007,5,22,1,27.1,12.1,2,88.0,65.0,77.0,59.0,66.0,0.0,12.0,421.0,1917.0,0.0,29.44,30.09,5.8,16.0,7.4,23.0
4,2007-05-29,CULEX PIPIENS/RESTUANS,T050,41.919343,-87.694259,1,0,2007,5,22,1,21.5,15.6,2,88.0,65.0,77.0,59.0,66.0,0.0,12.0,421.0,1917.0,0.0,29.44,30.09,5.8,16.0,7.4,23.0


In [60]:
# Save data
df_combined.to_csv('../data/combined_clean.csv', index = False)